```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Ouput Layer](#output-layer)**

Originating story can be found here: [COR-681](https://logius-prd.jira.odc-noord.nl/browse/COR-681)

# **FLOW DIAGRAMS**

---

`ADD DIAGRAM HERE!`

Required steps:

1. `ADD REQUIRMENT STEP LIST HERE!`

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>OL</font>**: Output Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/DataFactory/Statics/pl_ProcessVaccinationCampaignMasterData.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL]'))
CREATE TABLE [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] VARCHAR(50) NULL,
	[DATE_OF_STATISTICS] VARCHAR(50) NULL,
	[CAMPAIGN] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] VARCHAR(255) NULL
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL]'))
CREATE TABLE [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME NULL,
	[DATE_OF_STATISTICS] DATETIME NULL,
	[CAMPAIGN] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] INT NULL	-- ADD COLUMNS
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_RIVM_VACCINATION_CAMPAIGNS_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL] (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [CAMPAIGN],
        [TOTAL_VACCINATED]
    )
    SELECT
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_REPORT]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_STATISTICS]),
        UPPER(TRIM([CAMPAIGN])),
        [TOTAL_VACCINATED]
    FROM
        [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>RIVM VACCINATION CAMPAIGNS NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL]'))
CREATE TABLE [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME NULL,
	[DATE_OF_STATISTICS] DATETIME NULL,
	[CAMPAIGN] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] INT NULL,
	[TOTAL_VACCINATED_7DAYS] INT NULL,
	[CUMSUM_TOTAL_VACCINATED] INT NULL,
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_VACCINATION_CAMPAIGNS_NL] 
	ON [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_VACCINATION_CAMPAIGNS_NL]
    ON [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_REPEATING_SHOTS_ADMINISTERED_N]
AS
BEGIN
WITH CTE AS (
        SELECT 
			[ID],
            [DATE_OF_REPORT],
			[DATE_OF_STATISTICS],
            [CAMPAIGN],
            [TOTAL_VACCINATED]
        FROM [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL])
    ),
	CTE_7DAYS AS (
		SELECT 
			r.Id, 
			v.total_vaccinated
        FROM CTE r
		LEFT JOIN CTE v 
			ON r.CAMPAIGN = v.CAMPAIGN 
			AND v.DATE_OF_STATISTICS >= dateadd(day, -7, [dbo].[WEEK_START](r.[DATE_OF_STATISTICS])) and v.DATE_OF_STATISTICS < [dbo].[WEEK_START](r.[DATE_OF_STATISTICS]) 
	),
	CTE_7DAYS_SUM AS (
		SELECT Id AS Id, sum(total_vaccinated) AS SUM_VACCINATED
		FROM CTE_7DAYS
		group by Id
	),
	CTE_ALLDAYS AS (
		SELECT 
			r.Id, 
			v.total_vaccinated
        FROM CTE r
		LEFT JOIN CTE v 
			ON r.CAMPAIGN = v.CAMPAIGN 
			AND v.DATE_OF_STATISTICS < [dbo].[WEEK_START](r.[DATE_OF_STATISTICS]) 
	),
	CTE_ALLDAYS_SUM AS (
		SELECT Id AS Id, sum(total_vaccinated) AS SUM_VACCINATED
		FROM CTE_ALLDAYS
		group by Id
	)
    INSERT INTO [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] (
            [DATE_OF_REPORT],
			[DATE_OF_STATISTICS],
            [CAMPAIGN],
            [TOTAL_VACCINATED],
            [CUMSUM_TOTAL_VACCINATED],
            [TOTAL_VACCINATED_7DAYS]
    )
    SELECT
        c.[DATE_OF_REPORT],
        c.[DATE_OF_STATISTICS],
		c.[CAMPAIGN],
        c.[TOTAL_VACCINATED],
		c7.SUM_VACCINATED AS TOTAL_VACCINATED_7DAYS,
		ca.SUM_VACCINATED AS CUMSUM_TOTAL_VACCINATED
    FROM CTE c
	INNER JOIN CTE_7DAYS_SUM c7 ON c.ID = c7.Id
	INNER JOIN CTE_ALLDAYS_SUM ca ON c.Id = ca.Id
	

END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_REPEATING_SHOTS_ADMINISTERED_NL] AS
WITH CTE AS (
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [CAMPAIGN],
        [TOTAL_VACCINATED],
        [CUMSUM_TOTAL_VACCINATED],
        [TOTAL_VACCINATED_7DAYS]
    FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] WITH(NOLOCK))
)
SELECT
        c.[DATE_OF_REPORT],
        c.[DATE_OF_STATISTICS],
        c.[TOTAL_VACCINATED],
        c.[CUMSUM_TOTAL_VACCINATED],
        c.[TOTAL_VACCINATED_7DAYS],
        r.[CAMPAIGN_LABEL_NL],
        r.[CAMPAIGN_LABEL_EN],
        r.[CAMPAIGN_CODE],
        r.[ORDER]
FROM CTE c
INNER JOIN [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] r ON c.CAMPAIGN = r.[CAMPAIGN_CODE]
WHERE c.CAMPAIGN = 'n_pers_herhaalprik'
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_REPEATING_SHOTS_ADMINISTERED_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'repeating_shots_administered',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

## **<span style='color:teal'>BOOSTER SHOTS ADMINISTERED NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_BOOSTER_SHOTS_ADMINISTERED_NL] AS
WITH CTE AS (
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [CAMPAIGN],
        [TOTAL_VACCINATED],
        [CUMSUM_TOTAL_VACCINATED],
        [TOTAL_VACCINATED_7DAYS]
    FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] WITH(NOLOCK))
)
SELECT
        c.[DATE_OF_REPORT],
        c.[DATE_OF_STATISTICS],
        c.[TOTAL_VACCINATED],
        c.[CUMSUM_TOTAL_VACCINATED],
        c.[TOTAL_VACCINATED_7DAYS],
        r.[CAMPAIGN_LABEL_NL],
        r.[CAMPAIGN_LABEL_EN],
        r.[CAMPAIGN_CODE],
        r.[ORDER]
FROM CTE c
INNER JOIN [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] r ON c.CAMPAIGN = r.[CAMPAIGN_CODE]
WHERE c.CAMPAIGN = 'n_pers_booster1'
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_BOOSTER_SHOTS_ADMINISTERED_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'booster_shots_administered',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

## **<span style='color:teal'>PRIMARY SHOTS ADMINISTERED NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_PRIMARY_SHOTS_ADMINISTERED_NL] AS
WITH CTE AS (
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [CAMPAIGN],
        [TOTAL_VACCINATED],
        [CUMSUM_TOTAL_VACCINATED],
        [TOTAL_VACCINATED_7DAYS]
    FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] WITH(NOLOCK))
)
SELECT
        c.[DATE_OF_REPORT],
        c.[DATE_OF_STATISTICS],
        c.[TOTAL_VACCINATED],
        c.[CUMSUM_TOTAL_VACCINATED],
        c.[TOTAL_VACCINATED_7DAYS],
        r.[CAMPAIGN_LABEL_NL],
        r.[CAMPAIGN_LABEL_EN],
        r.[CAMPAIGN_CODE],
        r.[ORDER]
FROM CTE c
INNER JOIN [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] r ON c.CAMPAIGN = r.[CAMPAIGN_CODE]
WHERE c.CAMPAIGN = 'n_pers_basisserie'
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_PRIMARY_SHOTS_ADMINISTERED_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'primary_shots_administered',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_START_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO